data augmentation

Data augmentation is a technique used in machine learning and deep learning to artificially increase the size of a training dataset by applying various transformations to the existing data. The goal is to enhance the model's generalization ability, robustness, and performance by exposing it to a more diverse set of examples during training. This technique is particularly useful when the available dataset is limited.

Common data augmentation techniques include:

1. **Rotation:** Rotating images by a certain degree.
2. **Flip:** Flipping images horizontally or vertically.
3. **Zoom:** Zooming in or out of images.
4. **Translation:** Shifting images horizontally or vertically.
5. **Shear:** Applying a shearing transformation to images.
6. **Brightness and Contrast Adjustment:** Changing the brightness and contrast of images.
7. **Noise Injection:** Adding random noise to images.
8. **Color Jittering:** Adjusting the color values of images.

For example, in image classification tasks, if you have a dataset of cat images, you can apply data augmentation to generate variations of the same images by rotating, flipping, zooming, or changing their colors. This way, the model sees different perspectives of the same data during training, which can help it become more robust to variations and improve its performance on new, unseen data.

In deep learning frameworks and libraries, there are often built-in tools or functions for applying data augmentation during the training process. This allows practitioners to easily incorporate these transformations into their pipelines without manually creating augmented datasets.

In [3]:
from collections import defaultdict 
import numpy as np
import pandas as pd
from PIL import Image
from io import BytesIO
import os

In [ ]:
# Load the dataset
loaded_data = np.load("my_dataset.npz")

# Access images and labels
loaded_images = loaded_data['images']

# Now you can use loaded_images and loaded_labels in your code
image_df=pd.DataFrame(loaded_images)
image_df.head()

In [4]:
#load info about the data set
images_info=pd.read_csv(r"C:\Users\EmiliaGachowetz-Gepp\Documents\New ML\unzipped\flickr_logos_27_dataset\flickr_logos_27_dataset_training_set_annotation.txt", sep='\s+', header=None)

#renaming the columns correspondingly:
new_column_names = ['Image ID', 'Label', 'Subset', 'x1', 'y1','x2', 'y2']
images_info.columns=new_column_names
images_info.head()

Image ID   Label  Subset   x1   y1   x2   y2
0   144503924.jpg  Adidas       1   38   12  234  142
1  2451569770.jpg  Adidas       1  242  208  413  331
2   390321909.jpg  Adidas       1   13    5   89   60
3  4761260517.jpg  Adidas       1   43  122  358  354
4  4763210295.jpg  Adidas       1   83   63  130   93

In [ ]:
image_array=image_df.values
some_pic=image_array[0]
some_pic_image=some_pic.reshape((227,227))

coordinate_df=images.iloc[:, [0] + list(range(-4, 0))]
coordinate_df.head()

In [ ]:
#function to augment pictures and save them in an extra folder
def rotate_image(image, angle):
    '''This function rotates the input image by a specifed angle and
    returns a rotated image'''
    rotated_image = image.rotate(angle)
    return rotated_image

def scale_image(image, scale_factor):
    '''This function scales an input image by a scale factor'''

    width, height = image.size
    new_size = (int(width * scale_factor), int(height * scale_factor))
    scaled_image = image.resize(new_size)
    return scaled_image

def crop_and_save_images(df, output_folder):
    '''And finallz, this function crops the images and
    then retursn the croppped, rotated and scaled images in a specified output folder'''

    modified_df = pd.DataFrame(columns=['ImageID', 'Operation', 'FilePath'])

    for index, row in images.iterrows():
        image_id = row['Image ID']
        x1, y1, x2, y2 = row['x1'], row['y1'], row['x2'], row['y2']
        

        # Open the image (assuming 'image_path' is the path to your image file)
        image_path = f"C:/Users/EmiliaGachowetz-Gepp/Documents/New ML/images/flickr_logos_27_dataset_images/{image_id}"
        original_image = Image.open(image_path)

        # Rotate the image (example: 45 degrees)
        rotated_image = rotate_image(original_image, angle=45)
        rotated_image.save(f"{output_folder}/image_{image_id}_rotated.jpg")

        # Scale the image (example: scale factor of 1.5)
        scaled_image = scale_image(original_image, scale_factor=1.5)
        scaled_image.save(f"{output_folder}/image_{image_id}_scaled.jpg")

        # Crop the image and save (example: cropping a region)
        cropped_image = original_image.crop((x1, y1, x2, y2))
        cropped_image.save(f"{output_folder}/image_{image_id}_cropped.jpg")

        # Update modified_df with unique identifiers and file paths
        modified_df = modified_df._append({'ImageID': image_id, 'Operation': 'Rotated',
                                          'FilePath': f"{output_folder}/image_{image_id}_rotated.jpg"}, ignore_index=True)
        modified_df = modified_df._append({'ImageID': image_id, 'Operation': 'Scaled',
                                          'FilePath': f"{output_folder}/image_{image_id}_scaled.jpg"}, ignore_index=True)
        modified_df = modified_df._append({'ImageID': image_id, 'Operation': 'Cropped',
                                          'FilePath': f"{output_folder}/image_{image_id}_cropped.jpg"}, ignore_index=True)

    return modified_df

# Example usage:
output_folder = r"C:\Users\EmiliaGachowetz-Gepp\Documents\New ML\images_aug"
modified_df = crop_and_save_images(images, output_folder)
print(modified_df)


In [11]:
import os
import numpy as np
import pandas as pd
from PIL import Image

# Specify the path to your directory containing image files
image_directory = r'C:\Users\EmiliaGachowetz-Gepp\Documents\New ML\images_aug'

# Get a list of all files in the directory
image_files = [f for f in os.listdir(image_directory) if os.path.isfile(os.path.join(image_directory, f))]

# Create an empty DataFrame
image_df = pd.DataFrame()

# Iterate through image files and append to the DataFrame
for index, image_file in enumerate(image_files):
    try:
        # Construct the full path to the image file
        image_path = os.path.join(image_directory, image_file)

        # Open the image using Pillow (PIL)
        img = Image.open(image_path)

        # Resize the image to 227x227 pixels using LANCZOS resampling filter
        img_resized = img.resize((227, 227), Image.LANCZOS)

        # Convert the resized image to a 1D NumPy array (flatten the image)
        img_array = np.array(img_resized.convert('L')).flatten()

        # Create a new DataFrame row with image data and file name
        new_row = pd.DataFrame([img_array], columns=[f'pixel_{i}' for i in range(len(img_array))])
        new_row['file_name'] = image_file  # Add a new column for file name

        # Append the new row to the image_df
        image_df = pd.concat([image_df, new_row], ignore_index=True)

    except Exception as e:
        # Handle errors (e.g., if the image cannot be processed)
        print(f"Error processing image at index {index}: {e}")

# Display the new DataFrame containing image data and file names
print(image_df)

KeyboardInterrupt: 

In [3]:
# Save the dataset using NumPy's .npz format
np.savez("my_dataset_aug.npz", images=image_df)

In [5]:
#now we need to give the augmented images their labels
image_df['actual ID'] = image_df['file_name'].str.extract(r'(\d+\.jpg)')
merged_df = pd.merge(image_df, images_info, left_on='actual ID', right_on='Image ID', how='inner')
merged_df

KeyError: 'file_name'

In [6]:
images_full = merged_df.drop(['file_name', 'actual ID'], axis=1)
images_full

NameError: name 'merged_df' is not defined

In [ ]:
# Save the dataset using NumPy's .npz format
np.savez("my_dataset_aug_full.npz", images=image_full)